# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [108]:
import pandas as pd
import pymongo

In [109]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [110]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_category = pd.read_json("./data/US_category_id.json")
df_youtube.head(2)


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [111]:
df_category=pd.json_normalize(df_category['items'])
df_category["id"]=df_category["id"].astype(int)
df_youtube=df_youtube.merge(df_category[['id','snippet.title']],left_on="category_id", right_on="id")
df_youtube = df_youtube.rename(columns={"snippet.title" : "category"})


### Importer les données

In [112]:
data = df_youtube.to_dict(orient="records")
collection.insert_many(data)

InsertManyResult([ObjectId('656fad1e06ece324842c7582'), ObjectId('656fad1e06ece324842c7583'), ObjectId('656fad1e06ece324842c7584'), ObjectId('656fad1e06ece324842c7585'), ObjectId('656fad1e06ece324842c7586'), ObjectId('656fad1e06ece324842c7587'), ObjectId('656fad1e06ece324842c7588'), ObjectId('656fad1e06ece324842c7589'), ObjectId('656fad1e06ece324842c758a'), ObjectId('656fad1e06ece324842c758b'), ObjectId('656fad1e06ece324842c758c'), ObjectId('656fad1e06ece324842c758d'), ObjectId('656fad1e06ece324842c758e'), ObjectId('656fad1e06ece324842c758f'), ObjectId('656fad1e06ece324842c7590'), ObjectId('656fad1e06ece324842c7591'), ObjectId('656fad1e06ece324842c7592'), ObjectId('656fad1e06ece324842c7593'), ObjectId('656fad1e06ece324842c7594'), ObjectId('656fad1e06ece324842c7595'), ObjectId('656fad1e06ece324842c7596'), ObjectId('656fad1e06ece324842c7597'), ObjectId('656fad1e06ece324842c7598'), ObjectId('656fad1e06ece324842c7599'), ObjectId('656fad1e06ece324842c759a'), ObjectId('656fad1e06ece324842c75

## Question 1
Récupérer toutes les vidéos de la chaîne Apple.

In [113]:
cur = collection.find({"channel_title":"Apple"})
for c in cur :
    print(c["title"])

iPhone X — Introducing iPhone X — Apple
iPhone X — Introducing iPhone X — Apple
iPhone X — Introducing iPhone X — Apple
iPhone X — Introducing iPhone X — Apple
iPhone X — Introducing iPhone X — Apple
Apple Watch Series 3 + Apple Music — Roll — Apple
Apple Watch Series 3 + Apple Music — Roll — Apple
Apple Watch Series 3 + Apple Music — Roll — Apple
iPhone 8 Plus — Portraits of Her  — Apple
iPhone 8 Plus — Portraits of Her  — Apple
iPhone 8 Plus — Portraits of Her  — Apple


## Question 2
Compter le nombre de catégories différentes

In [114]:
result = collection.aggregate([{"$group": {"_id":"$category","count":{"$sum":1}}}])
for res in result:
    print(f"Category : {res['_id']}, nombre de projet : {res['count']}")

Category : Travel & Events, nombre de projet : 48
Category : Comedy, nombre de projet : 755
Category : Gaming, nombre de projet : 82
Category : Howto & Style, nombre de projet : 869
Category : Entertainment, nombre de projet : 1601
Category : Pets & Animals, nombre de projet : 116
Category : Science & Technology, nombre de projet : 512
Category : Nonprofits & Activism, nombre de projet : 14
Category : Sports, nombre de projet : 410
Category : Shows, nombre de projet : 2
Category : Music, nombre de projet : 1250
Category : People & Blogs, nombre de projet : 882
Category : News & Politics, nombre de projet : 623
Category : Education, nombre de projet : 334
Category : Autos & Vehicles, nombre de projet : 116
Category : Film & Animation, nombre de projet : 378


## Question 3
Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

In [115]:
for document in collection.find({"tags" : {"$exists":True}}):
    tags = document["tags"].split("|")
    collection.update_one({"_id":document["_id"]}, {"$set":{"tags": tags}})

## Question 4
Récupérer les vidéos les plus vues.

In [119]:
current = collection.find().sort([("views",-1)]).limit(5)
for cur in current :
    print(cur["channel_title"])

ibighit
ibighit
ZaynVEVO
BETNetworks
BETNetworks


## Question 5
Compter le nombre moyen de vues en fonction de la catégorie.

In [125]:
result = collection.aggregate([{"$group": {"_id":"$category","average":{"$avg":"$views"}}}])
for res in result:
    print(f"Category : {res['_id']}, moyenne de vue : {res['average']}")

Category : Sports, moyenne de vue : 728434.3243902439
Category : Pets & Animals, moyenne de vue : 651404.3879310344
Category : Entertainment, moyenne de vue : 1154868.4397251718
Category : Science & Technology, moyenne de vue : 924730.548828125
Category : Howto & Style, moyenne de vue : 537665.1806674339
Category : Gaming, moyenne de vue : 681081.8170731707
Category : Travel & Events, moyenne de vue : 464041.0833333333
Category : Comedy, moyenne de vue : 1240073.294039735
Category : Film & Animation, moyenne de vue : 1039472.6560846561
Category : News & Politics, moyenne de vue : 540955.5569823436
Category : Education, moyenne de vue : 547582.4700598803
Category : Autos & Vehicles, moyenne de vue : 607693.8706896552
Category : People & Blogs, moyenne de vue : 971532.7573696146
Category : Music, moyenne de vue : 1176553.612
Category : Nonprofits & Activism, moyenne de vue : 1110334.2142857143
Category : Shows, moyenne de vue : 8492.5


## Question 6
Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [127]:
resultat = collection.aggregate([
    {"$group": {"_id": "$channel_title", "average_likes": {"$avg": "$likes"}}},
    {"$sort": {"average_likes": -1}},
    {"$limit": 5}
])

for doc in resultat:
    print(f"Chaîne : {doc['_id']}, Moyenne de likes: {doc['average_likes']}")

Chaîne : ZaynVEVO, Moyenne de likes: 1431683.0
Chaîne : ibighit, Moyenne de likes: 1371766.25
Chaîne : melanie martinez, Moyenne de likes: 911871.5
Chaîne : BETNetworks, Moyenne de likes: 769687.625
Chaîne : jypentertainment, Moyenne de likes: 758826.6666666666
